In [36]:
import sys
sys.path.insert(0, '../../')
from src.questions_construction.main import PLAN_LENGTHS, QUESTION_CATEGORIES
from src.questions_construction.domains import DOMAIN_NAMES

from src.analysis.model_performances import *
from src.common import *
import random

In [64]:
model_name = 'gemini' #'gpt-4o'
by_file_name = {}
for i in range(1,4):
    # try:
    with open(f'{PROJECT_PATH}/results.human_eval.free_answers/{model_name}/human_{i}.all.{model_name}.json') as f:
        data_id = f'human_{i}'
        data = json.load(f)
        by_file_name[data_id] = {d['id']: d for _category, d_ls in data.items() for d in d_ls}
        for _id, d in by_file_name[data_id].items():
            d['annotator'] = data_id
    # except:
    #     print(f'{model_name}.human_{i}')

In [65]:
ids_by_category = defaultdict(set)
for file_name, d_ls in by_file_name.items():
    for _id, d in d_ls.items():
        ids_by_category[d['question_category']].add(d['id'])

In [66]:
results_by_category = {}

# category_name = 'object_tracking'
total_scores = []
for category_name in QUESTION_CATEGORIES:
    category_scores = []
    for q_id in ids_by_category[category_name]:
        annotators_scores = []
        for annotator, data_by_id in by_file_name.items():
            annotators_scores.append(int(data_by_id[q_id]['human_score']))
        mean_score = np.mean(annotators_scores)
        if mean_score > 0.5:
            category_scores.append(1)
        else:
            category_scores.append(0)
    total_scores += category_scores
    mean = np.mean(category_scores)
    std = np.std(category_scores)
    sem = std / np.sqrt(len(category_scores))
    results_by_category[category_name] = {'mean': mean, 'std': std, 'sem': sem, 'scores': category_scores, 'n': len(category_scores)}
    
mean_total = np.mean(total_scores)
std_total = np.std(total_scores)
sem_total = std_total / np.sqrt(len(total_scores))
print(f'Total mean: {mean}, std: {std}, sem: {sem}')

Total mean: 0.45, std: 0.49749371855331, sem: 0.11124297730643494


In [67]:
for category_name, results in results_by_category.items():
    print(f'{category_name}: '+'${'+ f'{results["mean"]*100}'+'}_{'+ f'{round(results["sem"]*100, 2)}'+'}$')
print(f'TOTAL AVG: '+'${'+ f'{mean_total*100}'+'}_{'+ f'{round(sem_total*100, 2)}'+'}$')

object_tracking: ${80.0}_{8.94}$
fluent_tracking: ${20.0}_{8.94}$
state_tracking: ${0.0}_{0.0}$
action_executability: ${50.0}_{11.18}$
effects: ${10.0}_{6.71}$
numerical_reasoning: ${10.0}_{6.71}$
hallucination: ${45.0}_{11.12}$
composite: ${45.0}_{11.12}$
TOTAL AVG: ${32.5}_{3.7}$
